In [1]:
import time
import random
import pandas as pd
from tqdm import tqdm
import modin.pandas as pm


In [2]:

csv = "/track/119.Test_Modin.csv"

def calT(ptask):
    calTr = []

    time0 = time.time()
    df = ptask.read_csv(csv)
    calTr.append(time.time()-time0)
    print(0)

    time1 = time.time()
    df.isnull()
    calTr.append(time.time()-time1)
    print(1)

    time2 = time.time()
    df.apply(lambda x: x/x.mean())
    calTr.append(time.time()-time2)
    print(2)

    return calTr


result = []
for rows, cols in tqdm([
    [10000, 10000],
    [1000, 1000],
    [100, 100],
    [10, 10]
]):
    df = pd.DataFrame([
        [
            random.random() 
            for i in range(cols)
        ]
        for j in range(rows)
    ])

    df.to_csv(csv, index=None)

    result.append([rows, cols] + calT(pd) + calT(pm))



  0%|          | 0/4 [00:00<?, ?it/s]

0
1
2
0
1


 25%|██▌       | 1/4 [03:17<09:53, 197.77s/it]

2
0
1
2
0
1


 50%|█████     | 2/4 [03:21<02:47, 83.73s/it] 

2
0
1
2
0
1


 75%|███████▌  | 3/4 [03:22<00:45, 45.84s/it]

2
0
1
2
0
1


100%|██████████| 4/4 [03:22<00:00, 50.66s/it]

2


In [3]:
result = pd.DataFrame(
    result, columns=["Rows", "Cols", "pdT1", "pdT2", "pdT3", "pmT1", "pmT2", "pmT3"])
for n in [1, 2, 3]:
    result[f"upT{n}"] = result[f"pmT{n}"]/result[f"pdT{n}"]
result


,Rows,Cols,pdT1,pdT2,pdT3,pmT1,pmT2,pmT3,upT1,upT2,upT3
0,10000,10000,38.013483,0.069456,3.416770,12.672273,0.127133,3.290792,0.333363,1.830416,0.963130
1,1000,1000,0.246920,0.000634,0.333674,0.933746,0.118438,0.792660,3.781579,186.753383,2.375554
2,100,100,0.010047,0.000320,0.025977,0.359767,0.034956,0.264576,35.809440,109.087798,10.185133
3,10,10,0.003814,0.000074,0.002991,0.080243,0.005882,0.105738,21.037817,79.076923,35.355230
